# Naive Bayes


En este notebook se van a estar trabajando los modelos basados en Naive Bayes. Estos modelos son relativamente sencillos. ....

## Configuraciones iniciales

Cargamos las bibliotecas correspondientes a este modelo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from preprocessing import prepararSetDeDatos
from preprocessing import categoricalNBPreprocessing

## Carga y preparacion del set de datos

Cargamos ambos set de datos, y se los pasamos a la funcion que realiza el armado hecho para el TP1.

In [ ]:
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')

In [ ]:
X,y = prepararSetDeDatos(X,y)

## CategoricalNB

In [ ]:
X_categoricalNB = categoricalNBPreprocessing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_categoricalNB, y, test_size=0.25, random_state=0)

In [ ]:
categoricalNB = CategoricalNB()

In [ ]:
categoricalNB.fit(X_train, y_train)

Realizamos ahora las predicciones correspondientes con la parte de entrenamiento y la de pruebas, una vez hecho eso, mostramos las metricas obtenidas en cada uno.

In [ ]:
y_pred = categoricalNB.predict(X_train)
print(classification_report(y_train, y_pred, target_names=['No vuelve','Vuelve']))

EXPLICACION?????

In [ ]:
y_pred = categoricalNB.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

EXPLICACION???????

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
fig, ax = plt.subplots(dpi=100)
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["No volveria","Volveria"], xticklabels=[ "No Volveria","Volveria"], ax=ax)
ax.set_title("Matriz de confusion")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

EXPLICACION???????

Graficamos ahora la curva ROC para ambos casos.

In [ ]:
fpr_cnb_test,tpr_cnb_test,thresholds_cnb_test = roc_curve(y_test,categoricalNB.predict_proba(X_test)[:,1])
fpr_cnb_train,tpr_cnb_train,thresholds_cnb_train = roc_curve(y_train,categoricalNB.predict_proba(X_train)[:,1])

zero_test = np.argmin(np.abs(thresholds_cnb_test))
zero_train = np.argmin(np.abs(thresholds_cnb_train))

plt.plot(fpr_cnb_train,tpr_cnb_train,label="ROC Curve Categorical NB Train")
plt.plot(fpr_cnb_test,tpr_cnb_test,label="ROC Curve Categorical NB Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(fpr_cnb_test[zero_test],tpr_cnb_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)
plt.plot(fpr_cnb_train[zero_train],tpr_cnb_train[zero_train],'x',markersize=10,label="threshold zero train",fillstyle="none",c="k",mew=2)

plt.legend(loc=4)

EXPLICACION??????

In [ ]:
auc_cnb = roc_auc_score(y_test,categoricalNB.predict_proba(X_test)[:, 1])
print("AUC para categorical NB: {:.3f}".format(auc_cnb))

## MultinomialNB

## GaussianNB